In [1]:
"""mplchart stylemap (experimental)"""

from collections import Counter, ChainMap


class StyleMap:
    def __init__(self, mapping, counter: Counter = None):
        if not isinstance(mapping, ChainMap):
            mapping = ChainMap(mapping)
        if counter is None:
            counter = Counter()

        self.mapping = mapping
        self.counter = counter

    def merge(self, mapping, *, reset: bool = False):
        mapping = self.mapping.new_child(mapping)
        counter = Counter() if reset else self.counter
        return self.__class__(mapping, counter)

    def copy(self, *, reset: bool = True):
        mapping = self.mapping
        counter = Counter() if reset else self.counter
        return self.__class__(mapping, counter)

    def __repr__(self):
        cname = self.__class__.__name__
        return f"{cname}({self.mapping!r}, {self.counter!r})"

    def get_setting(self, key: str, default=None):
        value = self.mapping.get(key)

        if isinstance(value, list) and len(value):
            count = self.counter[key]
            value = value[count % len(value)]
            self.counter[key] += 1

        if value is not None:
            return value

        return default


In [2]:

stylesheet = {
    "candleup.color": "black",
    "candledown.color": "red",
    "sma.color" : ["blue", "green", "red"],
    "rsi.color" : "black"
}

stylemap = StyleMap(stylesheet)

print(stylemap)

StyleMap(ChainMap({'candleup.color': 'black', 'candledown.color': 'red', 'sma.color': ['blue', 'green', 'red'], 'rsi.color': 'black'}), Counter())


In [3]:
for key in ["rsi.color", "sma.color", "sma.color"]:
    value = stylemap.get_setting(key)
    print(key, repr(value))


rsi.color 'black'
sma.color 'blue'
sma.color 'green'


In [4]:
from collections import defaultdict

class Chart:
    def __init__(self, stylesheet):
        stylesheet = dict(stylesheet)
        self.stylemaps = defaultdict(lambda: StyleMap(stylesheet))

    def get_stylemap(self, ax):
        return self.stylemaps[ax]


chart = Chart(stylesheet=stylesheet)
chart.get_stylemap(None)

StyleMap(ChainMap({'candleup.color': 'black', 'candledown.color': 'red', 'sma.color': ['blue', 'green', 'red'], 'rsi.color': 'black'}), Counter())